In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
df = pd.read_csv('../Desktop/data_2024_2015.csv')
weather = pd.read_csv('../Desktop/weather_data.csv')
df = df.sort_values(by=['setmana_epidemiologica', 'any'])
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
df['data'] = df['data'].dt.date
weather['DATA_LECTURA'] = pd.to_datetime(weather['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
weather['DATA_LECTURA'] = weather['DATA_LECTURA'].dt.date
weather
# 32 = temperature
# 33 = humidity

,ID,CODI_VARIABLE,DATA_LECTURA,VALOR_LECTURA
0,CW322710211630,32,2021-10-27,14.5
1,DL320702221000,32,2022-02-07,15.0
2,DL330702221000,33,2022-02-07,52.0
3,UW322505221200,32,2022-05-25,19.1
4,UW332505221200,33,2022-05-25,66.0
...,...,...,...,...
65732693,XG321503211030,32,2021-03-15,15.0
65732694,XG331503211000,33,2021-03-15,44.0
65732695,XG331503211030,33,2021-03-15,39.0
65732696,WU320407211030,32,2021-07-04,25.8


In [5]:
# separate the weather by temperature and humidity
temperature = weather[weather['CODI_VARIABLE'] == 32]
temperature = temperature.drop(columns=['CODI_VARIABLE', 'ID'])

humidity = weather[weather['CODI_VARIABLE'] == 33]
humidity = humidity.drop(columns=['CODI_VARIABLE', 'ID'])
humidity

,DATA_LECTURA,VALOR_LECTURA
2,2022-02-07,52.0
4,2022-05-25,66.0
6,2020-03-14,100.0
8,2020-09-24,60.0
10,2021-10-27,79.0
...,...,...
65732689,2020-12-19,81.0
65732691,2020-06-18,68.0
65732694,2021-03-15,44.0
65732695,2021-03-15,39.0


In [6]:
# merge temperature, humidity and df using 'data' and 'DATA_LECTURA' as keys
df = df.merge(temperature, left_on='data', right_on='DATA_LECTURA', how='left')


: 

In [ ]:
# group by 
df_grouped = df.groupby(['data', 'setmana_epidemiologica', 'any', 'nom_regio', 'diagnostic', 'grup_edat'], as_index=False).agg({'casos': 'sum', 'poblacio': 'sum'})
df_grouped

,data,setmana_epidemiologica,any,nom_regio,diagnostic,grup_edat,casos,poblacio
0,01/01/2017,52,2016,Alt Pirineu i Aran,Altres IRA,0,1,42
1,01/01/2017,52,2016,Alt Pirineu i Aran,Altres IRA,20 a 24,1,404
2,01/01/2017,52,2016,Alt Pirineu i Aran,Altres IRA,45 a 49,1,215
3,01/01/2017,52,2016,Alt Pirineu i Aran,Altres IRA,80 o més,1,344
4,01/01/2017,52,2016,Alt Pirineu i Aran,Faringoamigdalitis,45 a 49,1,215
...,...,...,...,...,...,...,...,...
2469525,31/12/2024,1,2025,Terres de l'Ebre,Pneumònia,50 a 54,2,910
2469526,31/12/2024,1,2025,Terres de l'Ebre,Pneumònia,65 a 69,1,696
2469527,31/12/2024,1,2025,Terres de l'Ebre,Pneumònia,70 a 74,1,717
2469528,31/12/2024,1,2025,Terres de l'Ebre,Pneumònia,75 a 79,1,501


In [31]:
df_grouped['tasa_casos'] = df_grouped['casos'] / df_grouped['poblacio']
df_grouped = df_grouped.drop(columns=['poblacio'])
df_grouped

,data,setmana_epidemiologica,any,nom_regio,diagnostic,grup_edat,casos,tasa_casos
0,01/01/2017,52,2016,Alt Pirineu i Aran,Altres IRA,0,1,0.023810
1,01/01/2017,52,2016,Alt Pirineu i Aran,Altres IRA,20 a 24,1,0.002475
2,01/01/2017,52,2016,Alt Pirineu i Aran,Altres IRA,45 a 49,1,0.004651
3,01/01/2017,52,2016,Alt Pirineu i Aran,Altres IRA,80 o més,1,0.002907
4,01/01/2017,52,2016,Alt Pirineu i Aran,Faringoamigdalitis,45 a 49,1,0.004651
...,...,...,...,...,...,...,...,...
2469525,31/12/2024,1,2025,Terres de l'Ebre,Pneumònia,50 a 54,2,0.002198
2469526,31/12/2024,1,2025,Terres de l'Ebre,Pneumònia,65 a 69,1,0.001437
2469527,31/12/2024,1,2025,Terres de l'Ebre,Pneumònia,70 a 74,1,0.001395
2469528,31/12/2024,1,2025,Terres de l'Ebre,Pneumònia,75 a 79,1,0.001996


In [32]:
print(df_grouped['diagnostic'].unique())

['Altres IRA' 'Faringoamigdalitis' 'Grip' 'Pneumònia' 'Bronquiolitis'
 'Faringoamigdalitis estreptocòccica' 'Escarlatina' 'Impetigen' 'COVID-19']


In [33]:
def process_diagnostic(df_grouped, diagnostic, new_column_name):
    df_filtered = df_grouped[df_grouped['diagnostic'] == diagnostic]
    df_filtered = df_filtered.drop(columns=['diagnostic'])
    df_filtered = df_filtered.rename(columns={'casos': new_column_name})
    return df_filtered

diagnostics = {
    'COVID-19': 'casos_covid',
    'Bronquiolitis': 'casos_bronquio',
    'Faringoamigdalitis': 'casos_fa',
    'Faringoamigdalitis estreptococica': 'casos_fa_es',
    'Grip': 'casos_grip',
    'Impetigen': 'casos_imp',
    'Escarlatina': 'casos_escarlatina',
    'Pneumònia': 'casos_pneumonia',
    'Altres IRA': 'casos_ira'
}

processed_dfs = {}
for diagnostic, new_column_name in diagnostics.items():
    processed_dfs[diagnostic] = process_diagnostic(df_grouped, diagnostic, new_column_name)

# Access individual dataframes like this:
df_covid = processed_dfs['COVID-19']
df_bronquio = processed_dfs['Bronquiolitis']
df_fa = processed_dfs['Faringoamigdalitis']
df_fa_es = processed_dfs['Faringoamigdalitis estreptococica']
df_grip = processed_dfs['Grip']
df_imp = processed_dfs['Impetigen']
df_escarlatina = processed_dfs['Escarlatina']
df_pneumonia = processed_dfs['Pneumònia']
df_ira = processed_dfs['Altres IRA']

# save the dataframes to csv files
os.makedirs('datasets_IRA', exist_ok=True)
df_covid.to_csv('datasets_IRA/covid.csv', index=False)
df_bronquio.to_csv('datasets_IRA/bronquilitis.csv', index=False)
df_fa.to_csv('datasets_IRA/fraringoamig.csv', index=False)
df_fa_es.to_csv('datasets_IRA/faringoamig_estrepto.csv', index=False)
df_grip.to_csv('datasets_IRA/grip.csv', index=False)
df_imp.to_csv('datasets_IRA/impetigen.csv', index=False)
df_escarlatina.to_csv('datasets_IRA/escarlatina.csv', index=False)
df_pneumonia.to_csv('datasets_IRA/pneumonia.csv', index=False)
df_ira.to_csv('datasets_IRA/altres_ira.csv', index=False)

